In [9]:
import krakenex
from pykrakenapi import KrakenAPI
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import csv
api = krakenex.API()
kraken = KrakenAPI(api)

## Getting data

- Historical data saved in the XBTUSD csv. 
- Daily data from 2013 until Sept 2021
- Using Kraken API to pull last quarters data and append to full csv file


In [10]:
df = pd.read_csv("./data/XBTUSD_history.csv", header=None)

df = df.rename(columns={0:'date_unix', 1:'open',2:'high',3:'low', 4:'close', 5:'volume', 6:'trades'})

df['date'] = pd.to_datetime(df['date_unix']*1e3, unit='ms')

### Using API to query latest data

In [11]:
cols = ['date','open', 'high', 'low','close','volume','date_unix']

In [12]:
pair = 'BTCUSD'

### 1632873600 -> 30th sept 2021
latest_px = kraken.get_ohlc_data(pair, interval=1440, ascending = True, since=1632873600) 
latest_px = latest_px[0]

latest_px = latest_px.reset_index().rename(columns={'dtime':'date', 'time':'date_unix'})

### Merging all data and store to csv

In [13]:
out_df = df[cols].append(latest_px[cols]).sort_values('date').reset_index(drop=True)

In [14]:
out_df.to_csv("./data/XBTUSD_DAILY.csv", index=False)

# Visualisation

In [15]:
import matplotlib.pyplot as plt

In [16]:
# note: OHLC API endpoint returns 720 rows only.
# ohlc[0].to_csv('./data/BTCGBP_latest.csv')
# print(ohlc[0])

# adding 20SMA (simple moving average, 20 days)
out_df['20 SMA'] = out_df['close'].rolling(20).mean()

# plot
fig = go.Figure(data=[go.Candlestick(x = out_df.index,
                                    open = out_df['open'],
                                    high = out_df['high'],
                                    low = out_df['low'],
                                    close = out_df['close'],
                                    ),
                     go.Scatter(x=out_df.index, y=out_df['20 SMA'], line=dict(color='purple', width=1))])

fig.show()